In [523]:
import baltic as bt
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from datetime import datetime as dt
from datetime import timedelta, datetime
import time
#import pymc3
import math
import arviz as az
#from hpd import hpd
import scipy.stats as stats
from io import StringIO
import altair as alt
from altair import datum
alt.data_transformers.disable_max_rows()
import seaborn as sns

from zipfile import ZipFile
import scipy as sp


import sys, subprocess, glob, os, shutil, re, importlib
from subprocess import call
import imp


%matplotlib inline
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patheffects as path_effects
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
import matplotlib.colors as clr
from matplotlib import rc
import textwrap as textwrap
from textwrap import wrap

import warnings
warnings.filterwarnings('ignore')

from scipy.special import binom

In [432]:
## read in Nes from BEAST log files
def read_in_Ne_changes_mascot(log_file_path):
    
    Ne_skyline_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                #print(line)
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    Ne_column_indices = []   # list to store column indices
                    Nes_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "Ne." in col:
                            Ne_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in Ne_column_indices:
                        name = line.split("\t")[n]
                        #deme = name.split(".")[1]# the syntax here is "Ne_region.1" where region is deme and 1 is interval 1
                        interval = name.split(".")[1]
                       
                        Nes_key[n] = name
                        Ne_skyline_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    Ne_skyline_dict["sample"].append(sample)

                    for index in Ne_column_indices:
                        name = Nes_key[index]
                        Ne_skyline_dict[name].append(line.split("\t")[index])
                    
                
    return(Ne_skyline_dict)

In [433]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "Ne." in i:
            interval = i.split(".")[1]
            local_series = input_df[i].astype('float').to_numpy()
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = math.exp(mean_log)
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)
        
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_Ne_log":mean_log,"mean_Ne_linear":mean_linear, 
                                                   "median_Ne_log" : median_log, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50, "lower_hpd_linear_50":lower_hpd_linear_50,
                                                  })
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)

In [434]:
#calculating transmission rate
def generate_summary_diff_df(input_df):
    
    
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "Ne." in i:
            interval = i.split(".")[1]
            next_interval = int(interval)+6 #averaging  to reduce noise
            local_series = input_df[i].astype('float').to_numpy()
           
            try:
                new_df["Ne"+ ".diff." + str(interval)] = (52/6)*(np.log(input_df[i].astype("float")) - np.log(input_df["Ne"+"." + str(next_interval)].astype('float')))
            
            
            except KeyError:
                pass 
            
            
    return(new_df)

In [435]:
log_file_path = "../multitree_coalescent/results/multicoal_smoothed_multicoal_case_prior_la_clusters_with_metadata_01_22.log"


In [436]:
Ne_skyline = read_in_Ne_changes_mascot(log_file_path)

#burnin
Ne_df = pd.DataFrame.from_dict(Ne_skyline)
print(len(Ne_df))
Ne_df

burnin_percent = 0.3
print(len(Ne_df))
rows_to_remove = int(len(Ne_df)* burnin_percent) ## remove burnin
Ne_df = Ne_df.iloc[rows_to_remove:]

print(len(Ne_df))
Ne_df = Ne_df.reset_index()
Ne_df.head()

4449
4449
3115


,index,sample,Ne.1,Ne.2,Ne.3,Ne.4,Ne.5,Ne.6,Ne.7,Ne.8,...,Ne.130,Ne.131,Ne.132,Ne.133,Ne.134,Ne.135,Ne.136,Ne.137,Ne.138,Ne.139
0,1334,66700000,-0.4150492051091268,-1.6486474195806042,-0.21226787930151833,-1.0388731392194992,0.4503062332291473,1.6592135977126616,-0.6016601666415693,-1.4606176795731745,...,0.9934582850687185,-0.8799907487007589,-0.7749585974519368,-3.2692068618142867,-2.986583475609112,-3.4963680260578243,-4.07409175415696,-3.934615484141743,-3.6423136320674363,-2.1114730944822333
1,1335,66750000,-3.121420831197207,-2.9812271622183864,-2.0550811206424937,-0.757269839346789,-0.2548725711791453,0.013937288113646983,-0.22107789977662876,-1.6564319616023484,...,1.0665457118237456,-1.0460954831239502,-0.18642644004509076,-2.3443642886180838,-1.998287116130621,-3.922454808163583,-3.9296788852167945,-3.2653387459411807,-1.9873012899395701,-2.218165903320699
2,1336,66800000,-3.0621602559055874,-3.0442725264941397,-2.500179451779213,0.45805921972253605,2.1987529291897268,2.9419986751771994,0.5842782068408559,0.1663906530648381,...,0.5319908086922801,-1.4747410112197725,-0.07038254712550668,-1.8261312878512734,-4.070300009581117,-4.358258234061046,-4.775973198323807,-4.240421037483388,-3.420015389775883,-1.249489881244836
3,1337,66850000,-0.9267055027591731,-2.073725770925092,-0.11620036695987819,0.33775727722827353,0.8783187290142405,0.31105615533046505,1.9447586222774824,1.7575038750783716,...,0.8725293322635398,-0.18098673475097468,-0.43796290298533097,-2.338004102534577,-2.2876650472042104,-2.4427382846010706,-3.9964116106358096,-3.4587398068797413,-1.83126089072077,-1.2859032671393118
4,1338,66900000,-2.0291882239609826,-1.7167622668345137,-0.8820287533445242,0.6932068815822862,-0.23022994024680069,-0.2554833693127013,-0.09465608448414907,-1.1772066904063456,...,1.3682461341340664,-0.8871183348190032,-0.20536640847336876,-2.5795714745019165,-2.1548824289767636,-2.657604200818937,-2.3079800716059564,-3.2009902686235594,-2.5553150350999703,-2.33163191031504


In [437]:
exp_ne_df = Ne_df.copy()
# Select only numeric columns excluding the first two columns
numeric_columns = exp_ne_df.columns[2:]  # Exclude the first two columns

# Apply the exponential function only to numeric columns
exp_ne_df[numeric_columns] = exp_ne_df[numeric_columns].apply(pd.to_numeric, errors='coerce').apply(np.exp)


uninfectious_rate = 365/4.5

#taken from https://www.medrxiv.org/content/10.1101/2022.08.17.22278897v1.full.pdf
incubation_period = 365/8

#calculate transmission rate based on SEIR dynamics
ne_diff_summary = generate_summary_diff_df(exp_ne_df)
seir_growth_rate = ((ne_diff_summary*2 + uninfectious_rate + incubation_period)**2 - (incubation_period- uninfectious_rate)**2)/(4*incubation_period)
seir_growth_rate.head()

,Ne.diff.1,Ne.diff.2,Ne.diff.3,Ne.diff.4,Ne.diff.5,Ne.diff.6,Ne.diff.7,Ne.diff.8,Ne.diff.9,Ne.diff.10,...,Ne.diff.124,Ne.diff.125,Ne.diff.126,Ne.diff.127,Ne.diff.128,Ne.diff.129,Ne.diff.130,Ne.diff.131,Ne.diff.132,Ne.diff.133
0,85.660926,76.642673,125.862775,83.345837,138.710519,171.899422,81.023432,18.707761,9.055616,45.993493,...,162.070063,188.996510,174.121563,236.572794,216.519691,281.740062,245.384028,170.009287,163.675215,55.446318
1,25.136454,52.107234,49.532612,113.082763,72.506969,88.963484,86.019840,37.538955,19.480780,14.764513,...,152.194196,213.767669,147.256676,193.046733,192.471263,259.806307,242.485235,142.645289,129.804610,78.099220
2,15.216145,20.787718,35.446127,130.240117,192.598192,145.434117,100.775798,73.718242,41.626492,20.160804,...,152.097793,251.273931,134.080039,197.283430,281.984214,261.336611,255.278260,160.284600,180.221646,67.776413
3,25.557279,13.042297,84.722314,131.395968,149.151182,111.531433,155.621204,107.240723,31.264689,10.356956,...,182.223229,166.721282,144.177632,225.735463,198.976956,243.439118,237.353819,177.706965,117.849342,57.605042
4,40.700069,68.601073,113.607398,131.744685,114.013888,100.559047,97.401671,60.145943,15.317678,41.297570,...,156.267649,232.742578,159.363860,238.420377,195.581948,225.204623,191.861611,145.629575,146.775082,75.243399


In [461]:
ne_summary = generate_summary_df(Ne_df)
test = ne_summary
test['days'] = (test.interval.astype(int) -3 )*7
test['date'] = dt.strptime("2024-12-12",  "%Y-%m-%d") - test.days.map(timedelta)
test = test.reset_index()

#calculate three week rolling mean
columns = ["mean_Ne_linear", "upper_hpd_linear_50","lower_hpd_linear_50", "lower_hpd_linear", "upper_hpd_linear"]
for column in columns:
    # Create a new column for the moving average
    test[f'{column}_MA'] = test[column].rolling(3, min_periods =1).mean()



In [503]:
range_ = ['#EEC060', '#2664A5', '#A76BB1', "#EEA160", "#356D4C"]
line_ne = alt.Chart(test).mark_area(interpolate='monotone', opacity = 1, clip = True, color = "#7c5981").encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,tickCount = "month",  format="%B %Y")),
    alt.Y('lower_hpd_linear_50_MA',axis=alt.Axis(title="Effective Population Size (magenta)", grid=False),scale=alt.Scale(domain=(0, 60))),
    alt.Y2('upper_hpd_linear_50_MA' ),
    
).properties(
    width=900,
    height=300)

band_ne = alt.Chart(test).mark_area(interpolate='monotone', opacity = 0.5, clip = True, color = "#7c5981").encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False, format="%B %Y")),
    alt.Y('lower_hpd_linear_MA',axis=alt.Axis(title="Effective Population Size (magenta)", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_linear_MA' ),
    
).properties(
    width=900,
    height=300
)

line_ne  + band_ne


alt.LayerChart(...)

In [504]:
la_mpox_cases_df = pd.read_csv("../multitree_coalescent/data/monkeypox_data.csv")
la_mpox_cases_df = la_mpox_cases_df.dropna(how = "all").dropna(axis = "columns", how = "all")
la_mpox_cases_df = la_mpox_cases_df.rename(columns= {"Unnamed: 0": "date"}); la_mpox_cases_df.head()

,date,cases
0,2022-05-19,1
1,2022-05-20,0
2,2022-05-21,0
3,2022-05-22,0
4,2022-05-23,0


In [505]:
cases = alt.Chart(la_mpox_cases_df).mark_bar(interpolate='monotone', opacity = 0.5, size = 1).encode(
    alt.X('date:T', axis=alt.Axis(title="", tickCount = "month",grid=False, format="%B %Y")),
    alt.Y('cases',axis=alt.Axis(title="Cases (Blue)", grid=False))
).properties(
    width=900,
    height=300
)

(cases + line_ne).resolve_scale(y = "independent")

alt.LayerChart(...)

In [374]:
#switching to migration

In [375]:
def read_in_forward_migration_rates_mascot(log_file_path):
    
    mig_rates_dict = {"sample":[]}
    
    with open(log_file_path, "r") as infile:
        line_number = 0
        for line in infile:
            #print(line_number)
            line_number += 1
            if not line.startswith("#"):  # log combiner will sometimes put the entire xml at the start of the log file
                # use the first line to find the migration rate columns
                
            # use the first line to find the migration rate columns
                if "posterior" in line:
                    all_cols = line.split("\t")
                    mig_column_indices = []   # list to store column indices
                    mig_key = {}   # dictionary to store the column index to map to column name

                    for i in range(len(all_cols)):
                        col = all_cols[i]
                        if "immigrationRate" in col:
                            mig_column_indices.append(i)

                    # make an empty dictionary to store Nes and generate dictionary to convert index to name
                    for n in mig_column_indices:
                        name = line.split("\t")[n]
                        interval = name.split(".")[1]# the syntax here is "NeLog.state01" where 0 is deme and 1 is interval 1
                        #interval = name.split(".")[2]
                       
                        mig_key[n] = name
                        mig_rates_dict[name] = []


                # read in actual parameter estimates and store in dictionary
                else:
                    sample = line.split("\t")[0]
                    mig_rates_dict["sample"].append(sample)

                    for index in mig_column_indices:
                        name = mig_key[index]
                        mig_rates_dict[name].append(line.split("\t")[index])           
                
                
    return(mig_rates_dict)

In [376]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "immigrationRate." in i:
            interval = i.split(".")[1]
           # print(interval)
            #print(interval)
            #print(i)
            #next_interval = int(interval)+1
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_log = local_series.mean()
            median_log = np.median(local_series)
            mean_linear = math.exp(mean_log)
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            lower_hpd_linear_95 = math.exp(lower_hpd_log_95)
            upper_hpd_log_95 = hpd_95[1]
            upper_hpd_linear_95 = math.exp(upper_hpd_log_95)
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            lower_hpd_linear_50 = math.exp(lower_hpd_log_50)
            upper_hpd_log_50 = hpd_50[1]
            upper_hpd_linear_50 = math.exp(upper_hpd_log_50)

            
            try:
                local_df = pd.DataFrame.from_dict({"interval":interval, "mean_mig_log":mean_log,"mean_mig_linear":mean_linear, 
                                                   "median_mig_log" : median_log, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50,
                                                   "upper_hpd_linear":upper_hpd_linear_95,"lower_hpd_linear":lower_hpd_linear_95,
                                                   "upper_hpd_linear_50":upper_hpd_linear_50, "lower_hpd_linear_50":lower_hpd_linear_50,
                                                  })
                new_df = new_df.append(local_df)
                #print(new_df)
            except:
                pass
            
   
            

            
    return(new_df)

In [377]:
migration_rates_f = read_in_forward_migration_rates_mascot(log_file_path)
mig_df_f = pd.DataFrame.from_dict(migration_rates_f)

#remove burnin
burnin_percent = 0.3
print(len(mig_df_f))
rows_to_remove = int(len(mig_df_f)* burnin_percent)
mig_df_f = mig_df_f.iloc[rows_to_remove:]

print(len(mig_df_f))
mig_df_f = mig_df_f.reset_index()
mig_df_f.head()

4449
3115


,index,sample,immigrationRate.1,immigrationRate.2,immigrationRate.3,immigrationRate.4,immigrationRate.5,immigrationRate.6,immigrationRate.7,immigrationRate.8,...,immigrationRate.129,immigrationRate.130,immigrationRate.131,immigrationRate.132,immigrationRate.133,immigrationRate.134,immigrationRate.135,immigrationRate.136,immigrationRate.137,immigrationRate.138
0,1334,66700000,-1.4283361361286757,-0.46485542994196016,0.6085976830182345,-0.09508524571561049,-0.4878072571131238,-0.9579833162812611,-1.0571928323517727,-0.5220745287252839,...,2.493151405340156,2.1333669956526875,2.940099604557461,2.3599861169066028,2.5442182497189285,2.782769095293822,2.6763615846942153,2.237864358771005,2.858790529847559,2.743341411614092
1,1335,66750000,0.9373411360414724,1.294170581919156,1.0204309947648615,0.543059142800109,-0.04062008232317779,-1.6521575051898314,-1.1130049838630307,-1.585196946172903,...,2.8575957976515003,2.7104061585565105,2.98261332892712,2.882878632849613,2.0628633356800767,2.3579043235942074,2.0760533931371272,2.1676748192519955,2.1675601195821512,1.4754657378307108
2,1336,66800000,1.0483938359735634,0.6838521532420206,0.777687213417006,0.44830175177429316,0.5413564369163945,1.0211465501039911,-0.44864750279042986,-0.9843215226283081,...,2.5256035268011416,2.2848025329196777,2.482422116676094,1.8878854756418622,2.015647703599142,2.694092595324931,1.9936580118536162,1.522257338005721,0.6622229443290322,1.570888922251831
3,1337,66850000,-1.3291360147614155,-1.209228417618184,-1.1389887651938333,-0.27128686745303615,-0.42191840732051306,-0.4062120315291242,0.11899659448191773,-0.11653498280636129,...,2.461258550178066,2.5160203283179174,3.502378241069932,3.405345058482367,3.110513884281638,2.7549612299319,3.2862865771964156,2.80808179924075,2.8412315238288492,2.2266112343610778
4,1338,66900000,-1.0122046350453948,-0.6716735095728821,-0.34619350614397604,-0.2606446272992706,-0.8470858803141886,-0.9300928487428134,-0.7524168789514294,-1.1855275125306284,...,2.710677691321528,2.7593719829312406,2.9242813382761796,3.339462961048558,3.6741321038190624,3.366951839599576,2.7227945831369333,2.627521748626604,2.9929548032899196,2.7174227542460043


In [378]:
mig_summary = generate_summary_df(mig_df_f)
mig_summary.head()

,interval,mean_mig_log,mean_mig_linear,median_mig_log,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,upper_hpd_linear,lower_hpd_linear,upper_hpd_linear_50,lower_hpd_linear_50
0,1,-0.606085,0.545482,-0.538128,1.811774,-3.392809,0.483221,-1.178409,6.121298,0.033614,1.621288,0.307768
0,2,-0.598220,0.549789,-0.522949,1.734311,-3.142245,0.377117,-1.183036,5.665025,0.043186,1.458074,0.306347
0,3,-0.579932,0.559937,-0.506013,1.547813,-3.129088,0.301370,-1.225017,4.701176,0.043758,1.351710,0.293753
0,4,-0.552191,0.575687,-0.492920,1.618991,-2.790347,0.191129,-1.297190,5.047996,0.061400,1.210616,0.273299
0,5,-0.507751,0.601847,-0.426653,1.559658,-2.597044,0.423113,-1.010034,4.757193,0.074493,1.526707,0.364207


In [379]:
#format dates
test_mig = mig_summary
test_mig['days'] = (test_mig.interval.astype(int) -3 )*7
test_mig['date'] = dt.strptime("2024-12-12",  "%Y-%m-%d") - test_mig.days.map(timedelta)

In [380]:
#domain = ['CentralEurope', 'NorthAmerica', 'SouthAmerica' ,"SouthernEurope", "WesternEurope"]
range_ = ['#EEC060', '#2664A5', '#A76BB1', "#EEA160", "#356D4C"]
line_mig = alt.Chart(test_mig).mark_area(interpolate='monotone', opacity = 0.8).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,tickCount = "month",  format="%B %Y")),
    alt.Y('lower_hpd_linear_50',axis=alt.Axis(title="Immigration Rate (migration events/lineage/year)", grid=False)),#,scale=alt.Scale(domain=(0, 13))),
    alt.Y2('upper_hpd_linear_50' ),
    
).properties(
    width=900,
    height=300
)

band_mig = alt.Chart(test_mig).mark_area(interpolate='monotone', opacity = 0.2, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,tickCount = "month",  format="%B %Y")),
    alt.Y('lower_hpd_linear',axis=alt.Axis(title="Immigration Rate (migration events/lineage/year)", grid=False), scale=alt.Scale(domain=(0, 100))),
    alt.Y2('upper_hpd_linear' ),
    
).properties(
    width=900,
    height=300
)


mig_plot = (line_mig + band_mig)
mig_plot

alt.LayerChart(...)

In [381]:
## percentage of cases due to introductions

In [382]:
def generate_percent_intro_df(input_df):
    
    temp_df = pd.DataFrame()
    new_df = pd.DataFrame()
   
    for i in input_df.columns.tolist():
        if "immigrationRate" in i:
        
            interval = i.split(".")[1]
            #deme = i.split(".")[2]

            try:
                temp_df["total."+ str(interval)] = seir_growth_rate["Ne.diff." + str(interval)].astype("float") +  input_df[i].astype("float").apply(np.exp)
                new_df["intro.percent"+ "." + str(interval)] = input_df[i].astype("float").apply(np.exp).div(temp_df["total."+ str(interval)], axis = 0) 

            except KeyError: #this was added because not all regions have equal time periods for their epidemics and it was throwing an error everytime it had to switch deme
                pass 
                  
    return(new_df)

In [383]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_summary_df(input_df):
    
    
    new_df = pd.DataFrame()

    for i in input_df.columns.tolist():
        if "percent" in i:
            interval = i.split(".")[2]
            #interval = i.split(".")[3]
            local_series = input_df[i].astype('float').to_numpy()
            #print(local_series)
            mean_percent = local_series.mean()
            hpd_95 = az.hdi(local_series, 0.95)
            lower_hpd_log_95 = hpd_95[0]
            upper_hpd_log_95 = hpd_95[1]
            hpd_50 = az.hdi(local_series, 0.50)
            lower_hpd_log_50 = hpd_50[0]
            upper_hpd_log_50 = hpd_50[1]
            

            
            
            try:
                local_df = pd.DataFrame.from_dict({ "interval":interval, "mean_percent":mean_percent, 
                                                   "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                   "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                new_df = new_df.append(local_df)
            except:
                pass
            
    return(new_df)


In [384]:
percent_df = generate_percent_intro_df(mig_df_f)
percent_df.head()

,intro.percent.1,intro.percent.2,intro.percent.3,intro.percent.4,intro.percent.5,intro.percent.6,intro.percent.7,intro.percent.8,intro.percent.9,intro.percent.10,...,intro.percent.124,intro.percent.125,intro.percent.126,intro.percent.127,intro.percent.128,intro.percent.129,intro.percent.130,intro.percent.131,intro.percent.132,intro.percent.133
0,0.002791,0.008130,0.014392,0.010792,0.004407,0.002227,0.004270,0.030739,0.045308,0.011214,...,0.037834,0.036041,0.034593,0.056486,0.062515,0.041177,0.033264,0.100132,0.060774,0.186761
1,0.092207,0.065428,0.053041,0.014993,0.013070,0.002149,0.003805,0.005429,0.018795,0.016335,...,0.028506,0.025406,0.038835,0.097339,0.075573,0.062835,0.058385,0.121559,0.120983,0.091528
2,0.157896,0.087025,0.057849,0.011878,0.008843,0.018733,0.006296,0.005044,0.014866,0.039660,...,0.023875,0.017515,0.056541,0.057751,0.057935,0.045642,0.037056,0.069491,0.035356,0.099700
3,0.010251,0.022370,0.003765,0.005769,0.004378,0.005937,0.007186,0.008231,0.031925,0.072094,...,0.020686,0.028720,0.037642,0.050600,0.044449,0.045930,0.049570,0.157393,0.203581,0.280275
4,0.008850,0.007392,0.006188,0.005815,0.003746,0.003908,0.004815,0.005055,0.016280,0.006756,...,0.014872,0.015933,0.047016,0.040236,0.042615,0.062601,0.076040,0.113369,0.161185,0.343757


In [385]:
summary_percent_df = generate_summary_df(percent_df)
summary_percent_df = summary_percent_df.reset_index()

#making sure that any numbers >1 are excluded as outliers from stochastic noise
percent_df =pd.DataFrame(np.where(percent_df <1, percent_df, 1), columns=percent_df.columns )
percent_df =pd.DataFrame(np.where(percent_df >0, percent_df, 0), columns=percent_df.columns )

#format dates
summary_percent_df['days'] = (summary_percent_df.interval.astype(int)-3)*7 #the 3 adjustment is made due to the fact that we take the difference of Nes over three weeks 
summary_percent_df['date'] = dt.strptime("2024-12-12",  "%Y-%m-%d") - summary_percent_df.days.map(timedelta)

summary_percent_df.head()

,index,interval,mean_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50,days,date
0,0,1,0.030533,0.122644,0.000031,0.011332,0.000053,-14,2024-12-26
1,0,2,0.034382,0.156354,0.000043,0.014203,0.000064,-7,2024-12-19
2,0,3,0.039274,0.135330,0.000053,0.013729,0.000332,0,2024-12-12
3,0,4,0.023515,0.078248,0.000054,0.011263,0.000405,7,2024-12-05
4,0,5,0.016769,0.053004,0.000046,0.009123,0.000466,14,2024-11-28


In [386]:
columns = ["mean_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50"]
for column in columns:
    # Create a new column for the moving average
    summary_percent_df[f'{column}_MA'] = summary_percent_df[column].rolling(6, min_periods =1).mean()


In [387]:
line1 = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.9, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month",grid=False,  format="%b %Y")),
    alt.Y('lower_hpd_log_50_MA',title = "Percent of new cases due to introductions", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, .3))),
    alt.Y2('upper_hpd_log_50_MA' )
).properties(
    width=900,
    height=300
)

band = alt.Chart(summary_percent_df).mark_area(interpolate='monotone', opacity = 0.2, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month",grid=False,  format="%b %Y")),
    alt.Y('lower_hpd_log_95_MA',title = "Percent of new cases due to introductions", axis=alt.Axis( grid=False, format='%', ), scale=alt.Scale(domain=(0.0, .3))),
    alt.Y2('upper_hpd_log_95_MA' )
).properties(
    width=900,
    height=300
)


intro_plot = (line1 + band).resolve_scale(y= "shared") ; intro_plot

alt.LayerChart(...)

In [388]:
## calculate Rt

In [389]:
def generate_local_and_intro_rt(input_df):
    
    
    new_df = pd.DataFrame()
    incubation_period = 365/8
    uninfectious_rate = 365/4.5
    
    for i in input_df.columns.tolist():
        if "Ne" in i:
            interval = i.split(".")[2]
            new_df["rt" +  "." + str(interval)] = (1+ (input_df[i].astype("float") / uninfectious_rate)) * (1+ (input_df[i].astype("float") / incubation_period))
            

            
    return(new_df)

In [390]:
def generate_local_rt(input_df):
    
    
    new_df = pd.DataFrame()
    incubation_period = 365/8
    uninfectious_rate = 365/4.5
    
    for i in input_df.columns.tolist():
        if "Ne" in i:
            interval = i.split(".")[2]
            try:
                new_df["rt" + "." + str(interval)] = ((1+ (input_df[i].astype("float") / uninfectious_rate)) * (1+ (input_df[i].astype("float") / incubation_period))) *  (1- percent_df["intro.percent."+str(interval)].astype("float"))
                
            except KeyError:
                pass

            
    return(new_df)

In [391]:
# make a new dataframe that summarizes the 95% HPD estimate with mean for each deme and interval 
def generate_rt_summary_df(input_df):
    
    
    new_df = pd.DataFrame()
    count = 0
    for i in input_df.columns.tolist():
        #print(i)
        if "rt" in i:
            if count %1 == 0:
                interval = i.split(".")[1]
                #interval = i.split(".")[2]
                local_series = input_df[i].astype('float').to_numpy()
                mean_percent = local_series.mean()
                median_percent = np.median(local_series)

                hpd_95 = az.hdi(local_series, 0.95)
                lower_hpd_log_95 = hpd_95[0]
                upper_hpd_log_95 = hpd_95[1]
                hpd_50 = az.hdi(local_series, 0.50)
                lower_hpd_log_50 = hpd_50[0]
                upper_hpd_log_50 = hpd_50[1]

                try:
                    local_df = pd.DataFrame.from_dict({"interval":interval, "mean_percent":mean_percent, "median_percent":median_percent,
                                                       "upper_hpd_log_95":upper_hpd_log_95,"lower_hpd_log_95":[lower_hpd_log_95], 
                                                       "upper_hpd_log_50":upper_hpd_log_50,"lower_hpd_log_50":lower_hpd_log_50})
                    new_df = new_df.append(local_df)
                except:
                    pass
            count+=1

    return(new_df)

In [493]:
# a line to show an Rt of 1
one_line = alt.Chart(pd.DataFrame({'y': [1.0]})).mark_rule(strokeDash=[1,1]).encode(y='y').properties(
    width=900,
    height=300
)

In [393]:
#read in data and calculate based on posterior set of trees
rt_local_and_intro_df = generate_local_and_intro_rt(ne_diff_summary)
rt_local_df = generate_local_rt(ne_diff_summary)

#summarize data
summary_rt_local_and_intro_df = generate_rt_summary_df(rt_local_and_intro_df)
summary_rt_local_df = generate_rt_summary_df(rt_local_df)

summary_rt_local_and_intro_df.head()

,interval,mean_percent,median_percent,upper_hpd_log_95,lower_hpd_log_95,upper_hpd_log_50,lower_hpd_log_50
0,1,0.732149,0.653602,1.606908,-0.014321,0.776627,0.274286
0,2,0.604429,0.539601,1.330037,-0.084981,0.694181,0.266884
0,3,0.627739,0.570311,1.410492,-0.004618,0.716958,0.266055
0,4,0.779283,0.717577,1.609350,0.067944,0.909133,0.430917
0,5,0.953123,0.903762,1.744817,0.233741,1.080193,0.585391


In [394]:
summary_rt_local_and_intro_df['days'] = (summary_rt_local_and_intro_df.interval.astype(int) -3) *7
summary_rt_local_and_intro_df['date'] = dt.strptime("2024-12-12",  "%Y-%m-%d") - summary_rt_local_and_intro_df.days.map(timedelta)

summary_rt_local_df['days'] = (summary_rt_local_df.interval.astype(int) -3) *7
summary_rt_local_df['date'] = dt.strptime("2024-12-12",  "%Y-%m-%d") - summary_rt_local_df.days.map(timedelta)

summary_rt_local_and_intro_df["contribution"] = "Local + Introductions"
summary_rt_local_df["contribution"] = "Local"

In [395]:
columns = ["mean_percent","median_percent", "upper_hpd_log_95","lower_hpd_log_95", "upper_hpd_log_50", "lower_hpd_log_50"]
for column in columns:
    # Create a new column for the moving average
    summary_rt_local_and_intro_df[f'{column}_MA'] = summary_rt_local_and_intro_df[column].rolling(6, min_periods =1).mean()
    summary_rt_local_df[f'{column}_MA'] = summary_rt_local_df[column].rolling(6, min_periods =1).mean()


In [396]:
combined_rt_df = pd.concat([summary_rt_local_and_intro_df,summary_rt_local_df], ignore_index = True )

In [506]:
rt_line2 = alt.Chart(combined_rt_df).mark_area(interpolate='monotone', opacity = 0.7, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('lower_hpd_log_50_MA',axis=alt.Axis(title="Phylo-estimated Rt", grid=False)),
    alt.Y2('upper_hpd_log_50_MA' ),
    alt.Color("contribution")
).properties(
    width=900,
    height=300
)
rt_band2 = alt.Chart(combined_rt_df).mark_area(
    opacity=0.3, interpolate='monotone'
).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,format="%B %Y")),
    alt.Y('lower_hpd_log_95_MA', axis=alt.Axis(title="Phylo-estimated Rt", grid=False)),# scale=alt.Scale(domain=(0.6, 1.3))),
    alt.Y2('upper_hpd_log_95_MA'), 
    alt.Color("contribution")
).properties(
    width=900,
    height=300
)

stream_plot_rt = rt_line2  + one_line

stream_plot_rt

alt.LayerChart(...)

In [507]:
line_simple_intro = alt.Chart(combined_rt_df).mark_line(interpolate='monotone', opacity = 1, clip = True, strokeDash=[5,1]).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('mean_percent_MA',scale=alt.Scale(domain=[0.5, 2]),axis=alt.Axis(title="Phylo-estimated Rt", grid=False)),
    alt.Color("contribution", legend=alt.Legend(offset=-200))

).properties(
    width=900,
    height=300
)


line_rt = line_simple_intro+ one_line; line_rt

alt.LayerChart(...)

In [508]:
alt.layer(intro_plot, line_rt).resolve_scale(y= "independent")

alt.LayerChart(...)

In [509]:
alt.layer(mig_plot, line_rt).resolve_scale(y= "independent")

alt.LayerChart(...)

In [510]:
cases_ne = (cases + line_ne)#.resolve_scale(y= "shared"); cases_ne

In [511]:
alt.layer(cases_ne,line_rt).resolve_scale(y = "independent")

alt.LayerChart(...)

In [486]:
line_rt

alt.LayerChart(...)

In [404]:
## compare phylo rt to microsim Rt

In [405]:
sim_df = pd.read_csv("../data/results_sep13_2024.csv")
sim_df["week"] = sim_df.date.str.rsplit(" ", 0).str[-0].tolist()
sim_df = sim_df.dropna()
sim_df = sim_df.replace("#DIV/0!", 0)

sim_df["Model (FoI=1) R_e"]= sim_df["Model (FoI=1) R_e"].astype("float")
sim_df["Model (FoI=1.25) R_e"]= sim_df["Model (FoI=1.25) R_e"].astype("float")
sim_df["Model (FoI=1.5) R_e"]= sim_df["Model (FoI=1.5) R_e"].astype("float")

sim_df.head()

,date,Empirical # of diagnosed cases,Model (FoI=1) diagnosed cases,Model (FoI=1) incidence,Model (FoI=1) symp,Model (FoI=1) R_e,Model (FoI=1.25) diagnosed cases,Model (FoI=1.25) incidence,Model (FoI=1.25) symp,Model (FoI=1.25) R_e,Model (FoI=1.5) diagnosed cases,Model (FoI=1.5) incidence,Model (FoI=1.5) symp,Model (FoI=1.5) R_e,Model (FoI=2) diagnosed cases,Model (FoI=2) incidence,week
1,7/10/22 - 7/16/22,125,123,140,279,2.320442,123,140,279,2.320442,123,140,279,2.320442,123,140,7/10/22
2,7/17/22 - 7/23/22,185,179,249,392,2.677419,179,249,392,2.677419,179,249,392,2.677419,179,249,7/17/22
3,7/24/22 - 7/30/22,221,202,302,495,2.311224,202,302,495,2.311224,202,302,495,2.311224,202,302,7/24/22
4,7/31/22 - 8/6/22,264,226,391,619,2.369697,226,391,619,2.369697,226,391,619,2.369697,226,391,7/31/22
5,8/7/22 - 8/13/22,261,258,143,645,0.693053,258,143,645,0.693053,258,143,645,0.693053,258,143,8/7/22


In [406]:
columns = ["Model (FoI=1) R_e","Model (FoI=1.25) R_e", "Model (FoI=1.5) R_e"]
for column in columns:
    # Create a new column for the moving average
    sim_df[f'{column}_MA'] = sim_df[column].rolling(3, min_periods =1, center = True).mean()



In [407]:
sim_df.head()

,date,Empirical # of diagnosed cases,Model (FoI=1) diagnosed cases,Model (FoI=1) incidence,Model (FoI=1) symp,Model (FoI=1) R_e,Model (FoI=1.25) diagnosed cases,Model (FoI=1.25) incidence,Model (FoI=1.25) symp,Model (FoI=1.25) R_e,Model (FoI=1.5) diagnosed cases,Model (FoI=1.5) incidence,Model (FoI=1.5) symp,Model (FoI=1.5) R_e,Model (FoI=2) diagnosed cases,Model (FoI=2) incidence,week,Model (FoI=1) R_e_MA,Model (FoI=1.25) R_e_MA,Model (FoI=1.5) R_e_MA
1,7/10/22 - 7/16/22,125,123,140,279,2.320442,123,140,279,2.320442,123,140,279,2.320442,123,140,7/10/22,2.498931,2.498931,2.498931
2,7/17/22 - 7/23/22,185,179,249,392,2.677419,179,249,392,2.677419,179,249,392,2.677419,179,249,7/17/22,2.436362,2.436362,2.436362
3,7/24/22 - 7/30/22,221,202,302,495,2.311224,202,302,495,2.311224,202,302,495,2.311224,202,302,7/24/22,2.452780,2.452780,2.452780
4,7/31/22 - 8/6/22,264,226,391,619,2.369697,226,391,619,2.369697,226,391,619,2.369697,226,391,7/31/22,1.791325,1.791325,1.791325
5,8/7/22 - 8/13/22,261,258,143,645,0.693053,258,143,645,0.693053,258,143,645,0.693053,258,143,8/7/22,1.245723,1.245723,1.245723


In [408]:
long_sims_df = pd.melt(sim_df, id_vars='week', value_vars=["Empirical # of diagnosed cases_MA",'Model (FoI=1) R_e_MA', 'Model (FoI=1.25) R_e_MA', 'Model (FoI=1.5) R_e_MA'])
long_sims_df.week = pd.to_datetime(long_sims_df.week)



KeyError: "The following 'value_vars' are not present in the DataFrame: ['Empirical # of diagnosed cases_MA']"

In [ ]:
subset_sims_df = long_sims_df[long_sims_df.week >"2023-03-01"]
subset_sims_df.variable[subset_sims_df.variable == "Empirical # of diagnosed cases_MA"] = "Empirical"
subset_sims_df.variable[subset_sims_df.variable == "Model (FoI=1) R_e_MA"] = "Model (FoI = 1)"
subset_sims_df.variable[subset_sims_df.variable == "Model (FoI=1.25) R_e_MA"] = "Model (FoI = 1.25)"
subset_sims_df.variable[subset_sims_df.variable == "Model (FoI=1.5) R_e_MA"] = "Model (FoI = 1.5)"



In [ ]:
subset_sims_df

In [534]:
## reading in microsimulation results for 2023-2024
sim_df_2023_onwards = pd.read_csv("../data/microsimulation_results/scenarios_rt.csv")
start_date = datetime(2023, 3, 4)
sim_df_2023_onwards['week_date'] = sim_df_2023_onwards['week'].apply(lambda x: (start_date + timedelta(weeks=x)).strftime('%Y-%m-%d'))
sim_df_2023_onwards.week_date = pd.to_datetime(sim_df_2023_onwards['week_date'])
sim_df_2023_onwards = sim_df_2023_onwards.dropna()
#sim_original_2023 = sim_df_2023_onwards[["week_date", "To.Aware_S0"]]


columns = ["r_t_S6"]
for column in columns:
    # Create a new column for the moving average
    sim_df_2023_onwards[f'{column}_MA'] = sim_df_2023_onwards[column].rolling(3, min_periods =1, center = True).mean()



In [535]:
sim_df_2023_onwards

,week,r_t_S0,r_t_S1,r_t_S2,r_t_S3,r_t_S4,r_t_S5,r_t_S6,r_t_S7,r_t_S8,...,r_t_S12,r_t_S13,r_t_S14,r_t_S15,r_t_S16,r_t_S17,r_t_S18,r_t_S19,week_date,r_t_S6_MA
0,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2023-03-04,0.135000
1,1,0.06,0.10,0.08,0.12,0.15,0.35,0.27,0.17,0.10,...,0.08,0.12,0.22,0.22,0.23,0.12,0.32,0.27,2023-03-11,0.230000
2,2,0.00,0.03,0.04,0.35,0.70,0.00,0.42,0.00,0.13,...,0.31,0.00,0.17,0.00,0.00,0.03,0.67,0.00,2023-03-18,0.446667
3,3,0.00,0.09,0.00,0.20,0.25,0.67,0.65,0.00,0.19,...,0.51,0.14,0.38,0.00,0.39,0.00,0.98,0.19,2023-03-25,0.476667
4,4,0.00,0.06,0.00,0.00,0.00,0.32,0.36,0.00,0.17,...,0.00,0.00,0.13,0.00,0.00,0.00,0.73,0.00,2023-04-01,0.546667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,81,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.97,0.15,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2024-09-21,0.000000
82,82,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.92,0.18,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2024-09-28,0.000000
83,83,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.94,0.12,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2024-10-05,0.000000
84,84,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.92,0.18,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,2024-10-12,0.000000


In [538]:
cases_bar =  alt.Chart(sim_df_2023_onwards, title = "").mark_line(interpolate='monotone', opacity = 0.8 ,color = "#f58518", clip = True).encode(
    alt.X('week_date', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('r_t_S6',axis=alt.Axis(title="Diagnosed Cases", grid=False),stack = False),

    #alt.Color('variable:N',legend=alt.Legend(title= "Contribution",offset = 10, labelFontSize = 16, titleFontSize = 20)),


).properties(
    width=800,
    height=300
)#.transform_filter(datum.variable == "Empirical")
cases_bar

alt.Chart(...)

In [513]:
long_sims_df["label"] = "Model (FoI=1.5) R_e_MA"

NameError: name 'long_sims_df' is not defined

In [514]:
stream_graph_all =  alt.Chart(long_sims_df, title = "").mark_line(interpolate='monotone', opacity = 1 ,color = "#72b7b2", clip = True).encode(
    alt.X('week:T', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('value',axis=alt.Axis(title="Estimated Rt", grid=False, labelFontSize = 16),stack = False),

    color = alt.Color('label',scale= alt.Scale(domain = ["Model (FoI=1.5) R_e_MA"], range = ["#72b7b2"]), legend=alt.Legend(title= "", offset = -190, labelFontSize = 10, titleFontSize = 10),),


).properties(
    width=800,
    height=300
).transform_filter(datum.variable == "Model (FoI=1.5) R_e_MA")

NameError: name 'long_sims_df' is not defined

In [515]:
stream_graph_all 

NameError: name 'stream_graph_all' is not defined

In [516]:
case_rt = pd.read_csv("../data/case-rt-estimates_region.tsv", sep = "\t"); case_rt.head()
case_rt['label'] = "case-based Rt"

In [517]:
case_rt_plot_median = alt.Chart(case_rt, title = "").mark_line(interpolate='monotone', opacity = 0.7 ,color = "black", clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('median_R',axis=alt.Axis(title="Estimated Rt", grid=False),stack = False, scale=alt.Scale(domain=(0,2.5))),
    alt.Opacity('label',legend=alt.Legend(title= "", offset = -170, labelFontSize = 10, titleFontSize = 10)),


).properties(
    width=800,
    height=300
)

case_rt_plot_bounds = alt.Chart(case_rt, title = "").mark_area(interpolate='monotone', opacity = 0.5 ,color = "black", clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('R_upper_95',axis=alt.Axis(title="Estimated Rt", grid=False),stack = False, scale=alt.Scale(domain=(0, 2.5))),
    alt.Y2("R_lower_95")



).properties(
    width=800,
    height=300
)


case_rt_combined = case_rt_plot_median+ case_rt_plot_bounds

In [540]:
   (stream_plot_rt +  case_rt_combined).resolve_scale(color = "independent")

alt.LayerChart(...)

In [519]:
combined_rt_df.contribution[combined_rt_df.contribution == 'Local + Introductions'] = "Phylo Rt - Local + Introductions"

combined_rt_df.contribution[combined_rt_df.contribution == 'Local'] = "Phylo Rt - Local "

rt_line2 = alt.Chart(combined_rt_df).mark_area(interpolate='monotone', opacity = 0.4, clip = True).encode(
    alt.X('date:T', axis=alt.Axis(title="",tickCount = "month", grid=False,format="%B %Y")),
    alt.Y('lower_hpd_log_50_MA',axis=alt.Axis(title="", grid=False)),
    alt.Y2('upper_hpd_log_50_MA' ),
    alt.Color("contribution", legend = alt.Legend(title = ""))
).properties(
    width=850,
    height=300
)
rt_band2 = alt.Chart(combined_rt_df).mark_area(
    opacity=0.3, interpolate='monotone'
).encode(
    alt.X('date:T', axis=alt.Axis(title="", grid=False,format="%B %Y")),
    alt.Y('lower_hpd_log_95_MA', axis=alt.Axis(title="", grid=False)),# scale=alt.Scale(domain=(0.6, 1.3))),
    alt.Y2('upper_hpd_log_95_MA'), 
    alt.Color("contribution")
).properties(
    width=850,
    height=300
)

stream_plot_rt = rt_line2  + one_line

stream_plot_rt

alt.LayerChart(...)

In [520]:
combined_rt_df[(combined_rt_df.date >"2023-03-01")  & (case_rt.date <"2024-08-16")].mean_percent.mean()

0.9965986808578253

In [ ]:
case_rt[(case_rt.date >"2023-03-01") & (case_rt.date <"2024-08-16")].median_R.mean()

In [ ]:
case_rt[case_rt.date >"2023-03-01"].median_R.mean()